In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [52]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [53]:
# Load the model trained on unmirrored data to show how it is assymetric
no_mirror_model = keras.models.load_model('model_01.h5')

In [54]:
import pandas as pd
import numpy as np
from json_functions import make_player_list, create_example

players_list = make_player_list('very-big.json')

teamA = ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve']
teamB = ['EliGE', 'Stewie2K', 'Grim', 'FalleN', 'NAF']

example_AvB = create_example(team1=teamA, team2=teamB, players_list=players_list)
print("Team A in slot 1 vs team B in slot 2:")
pred_AvB = no_mirror_model.predict(np.asarray([example_AvB]))
print(pred_AvB.reshape(2, 5))

example_BvA = create_example(team1=teamB, team2=teamA, players_list=players_list)
print("Team B in slot 1 vs team A in slot 2:")
pred_BvA = no_mirror_model.predict(np.asarray([example_BvA]))
print(pred_BvA.reshape(2, 5))

Team A in slot 1 vs team B in slot 2:
[[1.5204343  1.2641668  1.0912144  0.9385684  0.7884336 ]
 [1.1753037  1.0936733  0.86529243 0.7218606  0.731142  ]]
Team B in slot 1 vs team A in slot 2:
[[1.1279236  1.1067467  0.796592   0.68875617 0.7446804 ]
 [1.5647084  1.1852388  1.123609   0.97567785 0.88403565]]


In [55]:
# Now lets quantify how differently the ratings were predicted based on which team was in which slot
teamA_slot1 = pred_AvB.reshape(2, 5)[0]
teamA_slot2 = pred_BvA.reshape(2, 5)[1]

teamB_slot1 = pred_BvA.reshape(2, 5)[0]
teamB_slot2 = pred_AvB.reshape(2, 5)[1]

from sklearn.metrics import mean_absolute_error

mae_teamA = mean_absolute_error(teamA_slot2, teamA_slot1)
mae_teamB = mean_absolute_error(teamB_slot2, teamB_slot1)

print("mean abslote error of team A:", mae_teamA)
print("mean abslote error of team B:", mae_teamB)



mean abslote error of team A: 0.057661615
mean abslote error of team B: 0.03515934


# Mirroring Games
Now we will rebuild the dataframe, except for every game in the json file, we will create 2 games to train from, with the teams swapping spots 

In [56]:
def create_df_mirrored_games(file_to_use):
    raw_dataset = pd.read_json(file_to_use)
    processed_data = []

    players_list = make_player_list(file_to_use)
    numPlayers = len(players_list)

    for index, game in raw_dataset.iterrows():

        if len(game.team1) > 5 or len(game.team2) > 5:
            continue
        roster_team1 = [0] * numPlayers
        roster_team2 = [0] * numPlayers
        rating_vector = [0]*10

        # populate the team 1 roster and performance vectors
        for i, player in enumerate(game.team1):
            player_index = players_list.index(player['name'])
            roster_team1[player_index] = 1
            rating_vector[i] = round(float(player['rating']), 3)
        # populate the team 2 roster and performance vectors
        for i, player in enumerate(game.team2):
            player_index = players_list.index(player['name'])
            roster_team2[player_index] = 1
            rating_vector[i+5] = round(float(player['rating']), 3)
        row = [roster_team1+roster_team2, rating_vector]
        processed_data.append(row)
        
        # now flip the teams around and add that to the training data
        rating_vector_mirrored = rating_vector[5:10]+rating_vector[0:5]
        row_mirrored = [roster_team2+roster_team1, rating_vector_mirrored]
        processed_data.append(row_mirrored)

    return players_list, pd.DataFrame(processed_data, columns=['rosters vector',
                                                               'rating vector'])


In [57]:
players_list_2, df = create_df_mirrored_games('very-big.json')

features, outputs = df["rosters vector"], df['rating vector']

features = pd.DataFrame(features.values.tolist(), index= df.index)
outputs = pd.DataFrame(outputs.values.tolist(), index= df.index)

print(np.shape(features))
print(np.shape(outputs))

(17488, 1068)
(17488, 10)


In [58]:
# Now we'll train a model on the mirrored games
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(1068, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	# model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [59]:
n_inputs, n_outputs = features.shape[1], outputs.shape[1]

In [60]:
model = get_model(n_inputs, n_outputs)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.33, random_state=42)

In [62]:
model.fit(X_train, y_train, verbose=1, epochs=4, batch_size=500)
print("Evaluation on test set:")
mae = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/4
24/24 [==============================] - 0s 5ms/step - loss: 0.5936
Epoch 2/4
24/24 [==============================] - 0s 5ms/step - loss: 0.2070
Epoch 3/4
24/24 [==============================] - 0s 5ms/step - loss: 0.1852
Epoch 4/4
24/24 [==============================] - 0s 6ms/step - loss: 0.1768
Evaluation on test set:
181/181 [==============================] - 0s 2ms/step - loss: 0.1929


In [63]:
print("Team A in slot 1 vs team B in slot 2:")
pred_AvB_mirrored = model.predict(np.asarray([example_AvB]))
print(pred_AvB_mirrored.reshape(2, 5))

print("Team B in slot 1 vs team A in slot 2:")
pred_BvA_mirrored = model.predict(np.asarray([example_BvA]))
print(pred_BvA_mirrored.reshape(2, 5))

Team A in slot 1 vs team B in slot 2:
[[1.497519   1.2167796  1.1184615  1.08505    0.8956812 ]
 [1.1729797  0.98609143 0.9398011  0.79466444 0.6785067 ]]
Team B in slot 1 vs team A in slot 2:
[[1.1979443  0.9588642  0.98878944 0.7377537  0.67680544]
 [1.4609585  1.2312431  1.1338887  0.96310514 0.91969156]]


In [67]:
# Now lets quantify how differently the ratings were predicted based on which team was in which slot
teamA_slot1_mir = pred_AvB_mirrored.reshape(2, 5)[0]
teamA_slot2_mir = pred_BvA_mirrored.reshape(2, 5)[1]

teamB_slot1_mir = pred_BvA_mirrored.reshape(2, 5)[0]
teamB_slot2_mir = pred_AvB_mirrored.reshape(2, 5)[1]

mae_teamA_mir = mean_absolute_error(teamA_slot2_mir, teamA_slot1_mir)
mae_teamB_mir = mean_absolute_error(teamB_slot2_mir, teamB_slot1_mir)

print("mean abslote error of team A trained on mirrored games:", round(mae_teamA_mir, 3), "non-mirrored:", round(mae_teamA, 3) )
print("mean abslote error of team B trained on mirrored games:", round(mae_teamB_mir, 3), "non-mirrored:", round(mae_teamB, 3))

mean abslote error of team A trained on mirrored games: 0.042 non-mirrored: 0.058
mean abslote error of team B trained on mirrored games: 0.032 non-mirrored: 0.035


It seems like mirroring the games slightly reduces this problem, but I should test on more than just one game

In [68]:
players_list == players_list_2

True

In [79]:
import random
num_players= len(players_list)

preds_AvB = []
preds_BvA = []
preds_AvB_mir = []
preds_BvA_mir = []

for i in range(0, 500):
    teamA = []
    teamB = []
    for j in range(0, 5):
        teamA.append( players_list[random.randint(0, num_players-1)] )
        teamB.append( players_list[random.randint(0, num_players-1)] )
    
    AvB = create_example(teamA, teamB, players_list)
    BvA = create_example(teamB, teamA, players_list)

    pred_AvB = no_mirror_model.predict(np.asarray([AvB]))[0].tolist()
    pred_BvA = no_mirror_model.predict(np.asarray([BvA]))[0].tolist()

    pred_AvB_mirrored = model.predict(np.asarray([AvB]))[0].tolist()
    pred_BvA_mirrored = model.predict(np.asarray([BvA]))[0].tolist()

    preds_AvB.extend(pred_AvB)
    preds_BvA.extend(pred_BvA[5:10]+pred_BvA[0:5])

    preds_AvB_mir.extend(pred_AvB_mirrored)
    preds_BvA_mir.extend(pred_BvA_mirrored[5:10]+pred_BvA_mirrored[0:5])

err_mir = mean_absolute_error(preds_AvB_mir, preds_BvA_mir)
err = mean_absolute_error(preds_AvB, preds_BvA)
# print("error mirrored:", err_mir)
# print("error non-mirrored:", err)

print(preds_AvB[0:20])
print(preds_BvA[0:20])

[1.2996206283569336, 0.8413699269294739, 0.9585452079772949, 0.6270332932472229, 0.5554747581481934, 0.9675542712211609, 0.9180113077163696, 0.6275364756584167, 0.6720717549324036, 0.5016393661499023, 1.269755482673645, 0.9563808441162109, 0.9189533591270447, 0.7656739950180054, 0.5631893873214722, 1.1734142303466797, 1.1531709432601929, 1.1774002313613892, 0.7385519742965698, 0.7036744356155396]
[1.1138906478881836, 0.8115992546081543, 0.751567006111145, 0.7906836271286011, 0.6118819713592529, 1.219137191772461, 1.0641460418701172, 1.0334954261779785, 0.7137414216995239, 0.6560619473457336, 1.1405861377716064, 0.8060382604598999, 1.0057042837142944, 0.6832747459411621, 0.7849017381668091, 1.1554176807403564, 1.2117640972137451, 1.0568759441375732, 0.8240067362785339, 0.8355814218521118]


In [80]:
print("error mirrored:", round(err_mir, 3))
print("error non-mirrored:", round(err, 3))

error mirrored: 0.115
error non-mirrored: 0.127


# Conclusion

The error, which in this case is just the average sum of the differences in predictions when a matchup is in either order in the model.predict call, is smaller on the mirrored model. Ideally we'd want the predictions for either team to be the same no matter which order the teams go in the predict function. The mirrored data still makes the predictions more similar than non-mirrored, but not by much and I'm not sure why

In [101]:
preds_AvB = []
preds_BvA = []
preds_AvB_mir = []
preds_BvA_mir = []

for i in range(0, 100):

    AvB = X_test.iloc[i].tolist()
    BvA = X_test.iloc[i].tolist()[num_players : 2*num_players]+X_test.iloc[i].tolist()[0 : num_players]

    pred_AvB = no_mirror_model.predict(np.asarray([AvB]))[0].tolist()
    pred_BvA = no_mirror_model.predict(np.asarray([BvA]))[0].tolist()

    pred_AvB_mirrored = model.predict(np.asarray([AvB]))[0].tolist()
    pred_BvA_mirrored = model.predict(np.asarray([BvA]))[0].tolist()

    preds_AvB.extend(pred_AvB)
    preds_BvA.extend(pred_BvA[5:10]+pred_BvA[0:5])

    preds_AvB_mir.extend(pred_AvB_mirrored)
    preds_BvA_mir.extend(pred_BvA_mirrored[5:10]+pred_BvA_mirrored[0:5])

err_mir = mean_absolute_error(preds_AvB_mir, preds_BvA_mir)
err = mean_absolute_error(preds_AvB, preds_BvA)


error mirrored: 0.05474535962939262
error non-mirrored: 0.08323495674133301


In [103]:
print("error mirrored:    ", round(err_mir, 3))
print("error non-mirrored:", round(err, 3))

error mirrored:     0.055
error non-mirrored: 0.083


The result turns out to more dramatic for "realistic" rather than randomly generated data